In [39]:
%matplotlib inline
## imports

# site
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import flopy

# std
from pathlib import Path

In [40]:
def read_array(rasterfile, masked=True,  band=1):
    with rasterio.open(rasterfile) as src:
        return src.read(band, masked=masked)

In [41]:
def read_3d_array(rasterfiles, stack_axis=0, masked=True):
    arrays = []
    for rasterfile in rasterfiles:
        arrays.append(read_array(rasterfile, masked=masked))
    return np.ma.stack(arrays, axis=stack_axis)

In [42]:
## input

# name
name = 'test_wel_brabant'

# workspace
workspace = Path(r'..\output\wel')

# grid dimensions
nlay = 19
nrow = 450
ncol = 601
delr = 250.
delc = 250.
                 
# data files
idomainfile = Path(r'..\data\boundary\ibound.tif')
topfile = r'..\data\topbot\RL{ilay:d}.tif'
botfile = r'..\data\topbot\TH{ilay:d}.tif'
sqfile = Path(r'..\data\wells\sq_list.csv')

In [43]:
# read idomain
idomain = read_array(idomainfile).filled(0.)
print(idomain.shape)

(450, 601)


In [44]:
# read tops
topfiles = (Path(topfile.format(ilay=i + 1)) for i in range(nlay))

tops = read_3d_array(topfiles)

# mask bad nodata values
tops = np.ma.masked_where(tops.mask | (tops < -9990.), tops)

# fill masked with zeros
tops = tops.filled(0.)

print(tops.shape)

(19, 450, 601)


In [45]:
# read bots
botfiles = (Path(botfile.format(ilay=i + 1)) for i in range(nlay))

bots = read_3d_array(botfiles)

# mask bad nodata values
bots = np.ma.masked_where(bots.mask | (bots < -9990.), bots)

# fill masked with zeros
bots = bots.filled(0.)

print(bots.shape)

(19, 450, 601)


In [46]:
# convert to top, botm
top = tops[0, :, :]
botm = []
for ilay in range(nlay):
    botm.append(bots[ilay, :, :])
    if (ilay + 1) < nlay:
        botm.append(tops[ilay + 1, :, :])
botm = np.ma.stack(botm)
print(botm.shape)

(37, 450, 601)


In [47]:
# idomain
idomain = np.broadcast_to(idomain, botm.shape)
print(idomain.shape)

(37, 450, 601)


In [48]:
# Affine transformation is a lineair transformation that preserves parallel lines
# To do: incorporate corner coordinates in DIS
# to row,col from x,y
fwd = rasterio.transform.from_origin(xllcorner, yllcorner + nrows*cellsize, cellsize, cellsize)

# to x,y from row,col
rev = ~fwd

print(fwd)

| 250.00, 0.00, 60000.00|
| 0.00,-250.00, 435000.00|
| 0.00, 0.00, 1.00|


In [49]:
# read sources data from csv file
sqs = pd.read_csv(sqfile)

# transform xy to row,col
rows, cols = rasterio.transform.rowcol(fwd, sqs['xcoordinate'], sqs['ycoordinate'])

# layer numbers & pumping rates
layernumbers = (sqs.loc[:, 'ilay'] - 1).tolist()  # convert to zero based index
pumping_rates = sqs.loc[:, 'q_assigned'].tolist()
print(layernumbers[:10])

[0, 0, 9, 9, 9, 10, 10, 13, 13, 13]


In [50]:
# convert source data to stress period data records
stress_period_data = {}
stress_period_rec = []
for lay, row, col, q in zip(layernumbers, rows, cols, pumping_rates):
    stress_period_rec.append(((lay*2, row, col), q))
stress_period_data[0] = stress_period_rec

In [51]:
# Create the Flopy simulation object
sim = flopy.mf6.MFSimulation(
    sim_name=name,
    exe_name='mf6', 
    version='mf6',
    sim_ws=str(workspace))

# Create the Flopy temporal discretization object
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim,
    pname='tdis',
    time_units='DAYS',
    nper=1, 
    perioddata=[(1.0, 1, 1.0)],
    )

# Create the Flopy groundwater flow (gwf) model object
model_nam_file = '{}.nam'.format(name)
gwf = flopy.mf6.ModflowGwf(sim,
    modelname=name, 
   model_nam_file=model_nam_file,
   )

Directory structure already exists for simulation path c:\dev\mf6brabant\mf6brabant\notebooks\..\output\wel


In [52]:
# create workspace directory
workspace.mkdir(exist_ok=True)

In [53]:
# initialize the DIS package
dis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis(gwf,
    pname='dis', nlay=(nlay*2 - 1),
    nrow=nrow, ncol=ncol,
    delr=delr, delc=delc,
    top=top, botm=botm,
    idomain=idomain,
    )

In [54]:
# write to package file
dis.write()

In [55]:
wel = flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(gwf,
    pname='wel', 
    stress_period_data=stress_period_data)

In [56]:
# write to package file
wel.write()

INFORMATION: maxbound in ('gwf6', 'wel', 'dimensions') changed to 3779 based on size of stress_period_data
